Copyright 2023 The TensorFlow Authors.


In [ ]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Tutorial de TFX Pipeline para un modelo de lenguaje colosal con el conjunto de datos diario de CNN

En este codelab, usamos KerasNLP para cargar un modelo de lenguaje colosal (LLM) previamente entrenado (modelo GPT-2) y ajustarlo a un conjunto de datos. El conjunto de datos que se usa en esta demostración es el conjunto de datos diario de CNN. Tenga en cuenta que GPT-2 se usa aquí sólo para demostrar el proceso de un extremo a otro; las técnicas y las herramientas introducidas en este codelab son potencialmente transferibles a otros modelos de lenguaje generativo como Google T5.

<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td><a target="_blank" href="https://www.tensorflow.org/tfx/tutorials/tfx/CSV_Downloader_Component"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
<td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/tfx/tutorials/tfx/CSV_Downloader_Component.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
<td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/tfx/tutorials/tfx/CSV_Downloader_Component.ipynb"><img width="32px" src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a></td>
<td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/tfx/tutorials/tfx/CSV_Downloader_Component.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar el bloc de notas</a></td>
</table></div>

# Antes de empezar

Colab ofrece diferentes tipos de tiempos de ejecución. Asegúrese de ir a **Tiempo de ejecución -&gt; Cambiar tipo de tiempo de ejecución** y elija el tiempo de ejecución del Acelerador de hardware de GPU (que debe tener &gt;12G de RAM del sistema y ~15G de RAM de GPU), ya que ajustará el modelo GPT-2.

# Preparación

En primer lugar, tenemos que instalar el paquete de Python para TFX.

## Actualización de pip

Para evitar actualizar Pip en un sistema cuando se ejecuta localmente, verifique que se esté ejecutando en Colab. Por supuesto, los sistemas locales se pueden actualizar por separado.

In [ ]:
try:
  import colab
  !pip install --upgrade pip
except:
  pass

## Instalación de TFX

TFX actualmente está experimentando problemas con Python 3.10 en Colab. Por lo tanto, si solo ejecuta el siguiente comando

```
!pip install -U tfx
```

para instalar tfx, **obtendrá un error**. Por lo tanto, siga el código a continuación.

In [ ]:
%%shell
update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.8 3
curl -O https://bootstrap.pypa.io/get-pip.py
python get-pip.py

In [ ]:
# 1) TFX relies on an old version of google-api-core so we let google-auth float
# for the install. We grep it out below:
!grep -v google-auth /etc/requirements.core.in > requirements.txt

# 2) httplib2 should be included in /etc/requirements.core.in but it's not for
# reasons. We ensure it's included:
!grep httplib2 /etc/requirements.user.in >> requirements.txt

# 3) google.colab package is not available as a wheel. We symlink that in so
# it's on the sys.path of Python 3.8:
!mkdir /usr/local/lib/python3.8/dist-packages/google
!ln -s /usr/local/lib/python3.10/dist-packages/google/colab /usr/local/lib/python3.8/dist-packages/google/colab

# Now with those pre-requisites out of the way:
!pip install tfx==1.13.0 -r requirements.txt

In [ ]:
!pip install keras_nlp

# Importaciones

Primero, eliminemos nuestras importaciones.

In [ ]:
from tensorflow import keras
from tfx.types import Channel
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

## Desinstalación de shapely

TODO(b/263441833) Esta es una solución temporal para evitar un ImportError. En última instancia, debería solucionarse admitiendo una versión reciente de Bigquery, en lugar de desinstalar otras dependencias adicionales.


In [ ]:
!pip uninstall shapely -y

## ¿Reinició el tiempo de ejecución?

Si está usando Google Colab, la primera vez que ejecute la celda anterior, debe hacer clic en el botón "REINICIAR TIEMPO DE EJECUCIÓN" o usar el menú "Tiempo de ejecución &gt; Reiniciar tiempo de ejecución ..." para reiniciar el tiempo de ejecución. Esto se debe a la forma en que Colab carga los paquetes.

Verifique las versiones de TensorFlow y TFX.

Revisemos las versiones de la biblioteca.

In [ ]:
import tensorflow as tf
print('TensorFlow version: {}'.format(tf.__version__))
from tfx import v1 as tfx
print('TFX version: {}'.format(tfx.__version__))

## Configuración de variables

Hay algunas variables que se utilizan para definir una canalización. Puede personalizar estas variables como desee. De forma predeterminada, todos los resultados de la canalización se generarán en el directorio actual.

# Descargador de CSV

Para que la canalización sea más eficiente y para que sea posible la automatización, es útil tener un componente que incluya un enlace de descarga al archivo CSV que se descargará. Además, un objetivo importante de la canalización de ML de producción de TFX es recopilar metadatos que contengan información sobre los componentes de la canalización, sus ejecuciones y los artefactos resultantes. En otras palabras, el propósito de los metadatos es analizar el linaje de los componentes de la canalización y los problemas de depuración, y el componente de descarga CSV ayudaría a los usuarios a registrar y hacer un seguimiento de la información sobre la fuente de los datos y los pasos de preprocesamiento que los datos han experimentado antes de entrar a la canalización. En esta sección, declaramos un nuevo artefacto llamado CSVdoc y desarrollamos un componente personalizado, CSV Downloader, que almacena información sobre el conjunto de datos y descarga el archivo CSV en el URI del artefacto CSVdoc.

In [ ]:
from tfx.types import artifact
from tfx import types

Property = artifact.Property
PropertyType = artifact.PropertyType

URL_PROPERTY = Property(type=PropertyType.STRING)
PATH_PROPERTY = Property(type=PropertyType.STRING)

class CsvDoc(types.Artifact):
  """ Artifact that contains the CSV dataset.

     - 'url' : saves the source of the original data.
     - 'path': saves the path to the CSV file.
  """

  TYPE_NAME = 'CsvDoc'
  PROPERTIES = {
      'url' : URL_PROPERTY,
      'path': PATH_PROPERTY,
  }

In [ ]:
from absl import logging
import requests
import os
import tfx.v1 as tfx
from tfx.dsl.component.experimental.decorators import component

@tfx.dsl.components.component
def CsvDownloaderComponent(
    url: tfx.dsl.components.Parameter[str],
    file_name: tfx.dsl.components.Parameter[str],
    saved_file: tfx.dsl.components.OutputArtifact[CsvDoc],
) -> None:
  response = requests.get(url)
  saved_file.url = url
  if response.status_code == 200:
    file_path = os.path.join(saved_file.uri, file_name)
    saved_file.path = file_path
    url_content = response.content
    with open(file_path, 'wb') as csv_file:
      csv_file.write(url_content)
    logging.info(f"CSV file saved successfully at {file_path}")
  else:
    raise Exception("CSV file failed to be saved.")

In [ ]:
downloader = CsvDownloaderComponent(
  url = 'https://drive.google.com/uc?id=1YdZsJlRafqxiNSl0nHQkwR7rzrNlN9LI&export=download', file_name ='testing_doc.csv')

In [ ]:
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
context = InteractiveContext()

In [ ]:
context.run(downloader, enable_cache = False)